导入需要使用的包

In [2]:
import torch 
import torchvision
import torch.nn as nn
import numpy as np
import torchvision.transforms as transforms

创建Tensor

In [3]:
x = torch.tensor(1., requires_grad=True)
w = torch.tensor(2., requires_grad=True)
b = torch.tensor(3., requires_grad=True)

## 自动梯度计算基础

创建一个最简单的计算图，y=wx+b

In [4]:
y = w * x + b

计算梯度，由于x,w,b的requires_grad=True，所以可以直接使用.backward()方法计算。

In [5]:
y.backward()

输出梯度：这里可以直接做导数处理：
$$
\frac{\Delta y}{\Delta x}=\frac{\Delta{(w*x+b)}}{\Delta x}=w=2\\
\frac{\Delta y}{\Delta w}=\frac{\Delta{(w*x+b)}}{\Delta w}=x=1\\
\frac{\Delta y}{\Delta b}=\frac{\Delta{(w*x+b)}}{\Delta b}=1\\
$$

In [6]:
print(x.grad)    # x.grad = 2 
print(w.grad)    # w.grad = 1 
print(b.grad)    # b.grad = 1

tensor(2.)
tensor(1.)
tensor(1.)


创建两个随机数的Tensor

In [7]:
x = torch.randn(10, 3)
y = torch.randn(10, 2)

创建一个全连接层，weights是随机的。

In [10]:
linear = nn.Linear(3, 2)
print ('w: ', linear.weight)
print ('b: ', linear.bias)

w:  Parameter containing:
tensor([[-0.1124, -0.5236, -0.1964],
        [ 0.4323, -0.0193, -0.3696]], requires_grad=True)
b:  Parameter containing:
tensor([-0.0984, -0.2579], requires_grad=True)


使用MSE作为损失函数，以及SGD优化器。损失函数与优化器是配合使用的，二者缺一不可。

In [11]:
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(linear.parameters(), lr=0.01)

使用全连接层前向计算,由于输入是10*3的，而weights是3*2的，所以输出的矩阵为10*2的，与y的size一致的，用输出pred与y做loss的计算。

In [13]:
pred = linear(x)
loss = criterion(pred, y)
print('loss: ', loss.item())

loss:  0.7255082130432129


完成一次计算后计算梯度：

In [15]:
loss.backward()
print ('dL/dw: ', linear.weight.grad) 
print ('dL/db: ', linear.bias.grad)

RuntimeError: Trying to backward through the graph a second time, but the buffers have already been freed. Specify retain_graph=True when calling backward the first time.

In [16]:
optimizer.step()

In [17]:
pred = linear(x)
loss = criterion(pred, y)
print('loss after 1 step optimization: ', loss.item())

loss after 1 step optimization:  0.7144414782524109


## 从Numpy载入数据

创建一个numpy array,并使用`torch.from_numpy()`转化为Tensor，也可以通过.numpy()转化为Numpy Array.

In [26]:
x=np.array([[1,2],[3,4]])
y=torch.from_numpy(x)
z=y.numpy()
print(type(y))
print(type(z))

<class 'torch.Tensor'>
<class 'numpy.ndarray'>


## 输入Pipeline

首先通过torchvision载入CIFAR10数据。

In [4]:
train_dataset = torchvision.datasets.CIFAR10(root='../../data/',
                                             train=True, 
                                             transform=transforms.ToTensor(),
                                             download=True)
image, label = train_dataset[0]
print (image.size())
print (label)

Files already downloaded and verified
torch.Size([3, 32, 32])
6


在数据训练的时候，是一个batch一个batch送入的，使用DataLoader比较方便。

In [5]:
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=64, 
                                           shuffle=True)

在`train_loader`的基础上生成一个迭代器，然后使用迭代器的.next()方法不断生成不同的数据组。

In [10]:
data_iter = iter(train_loader)
images, labels = data_iter.next()
num_present=10 # 显示10行
for images, labels in train_loader:
    # Training code should be written here.
    if num_present>0:
        print(images.size(),labels.size())
        num_present-=1
    else:
        break

torch.Size([64, 3, 32, 32]) torch.Size([64])
torch.Size([64, 3, 32, 32]) torch.Size([64])
torch.Size([64, 3, 32, 32]) torch.Size([64])
torch.Size([64, 3, 32, 32]) torch.Size([64])
torch.Size([64, 3, 32, 32]) torch.Size([64])
torch.Size([64, 3, 32, 32]) torch.Size([64])
torch.Size([64, 3, 32, 32]) torch.Size([64])
torch.Size([64, 3, 32, 32]) torch.Size([64])
torch.Size([64, 3, 32, 32]) torch.Size([64])
torch.Size([64, 3, 32, 32]) torch.Size([64])


我们自己数据需要按照这种方法使用的时候，需要继承`torch.utils.data.Dataset`创建一个新的数据类，类中的`__init__`方法，`__getitems__`方法，以及`__len__`方法需要重写。

In [13]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self):
        # TODO
        # 1. Initialize file paths or a list of file names. 
        pass
    def __getitem__(self, index):
        # TODO
        # 1. Read one data from file (e.g. using numpy.fromfile, PIL.Image.open).
        # 2. Preprocess the data (e.g. torchvision.Transform).
        # 3. Return a data pair (e.g. image and label).
        pass
    def __len__(self):
        # You should change 0 to the total size of your dataset.
        return 0 
    


然后创建Loader:
~~~
custom_dataset = CustomDataset()
train_loader = torch.utils.data.DataLoader(dataset=custom_dataset,
                                           batch_size=64, 
                                           shuffle=True)
~~~

## 迁移学习

可以通过`torchvision.models`调用常用的模型，并加载weights可以方便训练。

In [16]:
resnet = torchvision.models.resnet18(pretrained=True)

在默认情况下，加载的模型是可训练的，如果不需要训练，或者只训练部分层，将`.requires_grad`设置为False.

In [18]:
for param in resnet.parameters():
    param.requires_grad = False

替换top层为自己模型的参数，如这里使用10个输出。

In [20]:
resnet.fc = nn.Linear(resnet.fc.in_features, 10)

## 保存与加载模型

In [22]:
# 保存和加载整个模型.
torch.save(resnet, 'model.ckpt')
model = torch.load('model.ckpt')

# 只保存和加载weights.
torch.save(resnet.state_dict(), 'params.ckpt')
resnet.load_state_dict(torch.load('params.ckpt'))

<All keys matched successfully>